In [3]:
import pandas as pd
#from pattern3.text import parse
import matplotlib.pyplot as plt
import numpy as np
# from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import sent_tokenize,word_tokenize
import re
from collections import defaultdict
from nltk.corpus import stopwords
stp_wrds= stopwords.words('english')

In [8]:
df_train= pd.read_csv('hm_train.csv')
df_test= pd.read_csv('hm_test.csv')
df_sample = pd.read_csv('sample_submission.csv')

In [69]:
df_train.head()

,hmid,reflection_period,cleaned_hm,num_sentence,predicted_category
0,27673,24h,I went on a successful date with someone I fel...,1,affection
1,27674,24h,I was happy when my son got 90% marks in his e...,1,affection
2,27675,24h,I went to the gym this morning and did yoga.,1,exercise
3,27676,24h,We had a serious talk with some friends of our...,2,bonding
4,27677,24h,I went with grandchildren to butterfly display...,1,affection


In [71]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60321 entries, 0 to 60320
Data columns (total 5 columns):
hmid                  60321 non-null int64
reflection_period     60321 non-null object
cleaned_hm            60321 non-null object
num_sentence          60321 non-null int64
predicted_category    60321 non-null object
dtypes: int64(2), object(3)
memory usage: 2.3+ MB


In [93]:
df_train['predicted_category'].value_counts()

affection           20880
achievement         20274
bonding              6561
enjoy_the_moment     6508
leisure              4242
nature               1127
exercise              729
Name: predicted_category, dtype: int64

In [94]:
df_test.head()

,hmid,reflection_period,cleaned_hm,num_sentence
0,88305,3m,I spent the weekend in Chicago with my friends.,1
1,88306,3m,We moved back into our house after a remodel. ...,2
2,88307,3m,My fiance proposed to me in front of my family...,1
3,88308,3m,I ate lobster at a fancy restaurant with some ...,1
4,88309,3m,I went out to a nice restaurant on a date with...,5


In [95]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40213 entries, 0 to 40212
Data columns (total 4 columns):
hmid                 40213 non-null int64
reflection_period    40213 non-null object
cleaned_hm           40213 non-null object
num_sentence         40213 non-null int64
dtypes: int64(2), object(2)
memory usage: 1.2+ MB


# Preprocessing

In [15]:
df_train['cleaned_hm'] = df_train['cleaned_hm'].apply(lambda x: x.replace('\n',''))
df_train['cleaned_hm'] = df_train['cleaned_hm'].apply(lambda x: x.replace('\r',''))
# df_train['cleaned_hm'] = df_train['cleaned_hm'].apply(lambda x: x.replace("\\",''))

In [60]:

# import re
# from collections import defaultdict
# cleaned = defaultdict(list)
for hm in df_train['cleaned_hm']:
    g = sent_tokenize(hm)
    if len(g) == 1:
        for i in range(len(g)):
            cleaned['cleaned_hm'].append(g[i])
            b=int(re.search(r'\d+', str(df_train[df_train['cleaned_hm'] == hm]['predicted_category'])).group())
            cleaned['predicted_category'].append(df_train[df_train['cleaned_hm'] == hm]['predicted_category'][b])
#     else:
#         cleaned['cleaned_hm'].append(g)
#         b=int(re.search(r'\d+', str(df_train[df_train['cleaned_hm'] == hm]['predicted_category'])).group())
#         cleaned['predicted_category'].append(df_train[df_train['cleaned_hm'] == hm]['predicted_category'][b])
    

In [68]:
cleaned_df = pd.DataFrame(cleaned)

str

In [6]:
cleaned_df= pd.read_csv('cleaned.csv', index_col=[0])

In [26]:
from nltk.corpus import stopwords
stp_wrds= stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stp_wrds])
    return sen_new
wo_stpwrds = [remove_stopwords(r.split()) for r in cleaned_df['cleaned_hm']]

In [30]:
wo_stpwrds[0]

'We serious talk friends flaky lately.'

In [31]:
df_train['cleaned_hm'][3]

'We had a serious talk with some friends of ours who have been flaky lately. They understood and we had a good evening hanging out.'

In [38]:
wo_stp = pd.DataFrame()
wo_stp['cleaned_hm'] = wo_stpwrds
wo_stp['predicted_category'] = cleaned_df['predicted_category']

In [39]:
wo_stp.isna().sum()

cleaned_hm            0
predicted_category    0
dtype: int64

In [96]:
cleaned_df.to_csv('cleaned.csv')

# Models

In [42]:
lb_train= LabelEncoder().fit_transform(df_train['predicted_category'])

In [39]:
df_train['predicted_category_labelled'] = lb

In [41]:
df_train.predicted_category.unique()

array(['affection', 'exercise', 'bonding', 'leisure', 'achievement',
       'enjoy_the_moment', 'nature'], dtype=object)

# Naive Bayes

In [45]:
# from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


X_train, X_test, y_train, y_test = train_test_split(df_train['cleaned_hm'], df_train['predicted_category'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [42]:
from sklearn.naive_bayes import MultinomialNB
mnb= MultinomialNB()

In [76]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(mnb, X_train_tfidf, df_train['predicted_category'], cv=10)
scores

array([0.71151616, 0.7136228 , 0.71788497, 0.71672468, 0.72115385,
       0.70971485, 0.7035809 , 0.69557287, 0.70364842, 0.69961851])

In [120]:
scores = cross_val_score(mnb, X_train_tfidf, df_train['predicted_category'], cv=5, scoring='f1_weighted')
scores

c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\adnan\a

array([0.65708749, 0.66470331, 0.65553614, 0.63236902, 0.63735649])

In [80]:
from sklearn.model_selection import ShuffleSplit
>>> n_samples = df_train.shape[0]
>>> cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
>>> cross_val_score(mnb, X_train_tfidf, df_train['predicted_category'], cv=cv)

array([0.69298779, 0.70044759, 0.69564016, 0.69834779, 0.69978449])

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(wo_stp['cleaned_hm'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [43]:
from sklearn.model_selection import StratifiedShuffleSplit
n_samples = wo_stp.shape[0]
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(mnb, X_train_tfidf, wo_stp['predicted_category'], cv=cv)


array([0.67449389, 0.67383821, 0.66568314, 0.67146136, 0.66769117])

# Logistic Regression


In [35]:
from sklearn.linear_model import LogisticRegression
lr =LogisticRegression()

In [125]:
g= lr.predict(count_vect.transform(df_test['cleaned_hm']))

In [126]:
result = pd.DataFrame()

In [127]:
result['hmid'] = df_test['hmid']
result['predicted_category'] = g

In [128]:
result.head()

,hmid,predicted_category
0,88305,bonding
1,88306,affection
2,88307,affection
3,88308,bonding
4,88309,bonding


In [129]:
result.to_csv('result_LR_cln.csv')

In [100]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lr, X_train_tfidf, cleaned_df['predicted_category'], cv=5)
scores

c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


array([0.6712354 , 0.72000246, 0.86113843, 0.89746112, 0.88200935])

In [102]:

scores = cross_val_score(lr, X_train_tfidf, cleaned_df['predicted_category'], cv=5, scoring='f1_weighted')
scores

c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


array([0.66660429, 0.71237142, 0.85743778, 0.89346259, 0.87456242])

In [104]:
from sklearn.model_selection import ShuffleSplit
n_samples =cleaned_df.shape[0]
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(lr, X_train_tfidf, cleaned_df['predicted_category'], cv=cv)

array([0.82079338, 0.81640849, 0.81698221, 0.81899025, 0.82317023])

In [40]:
from sklearn.model_selection import StratifiedShuffleSplit
n_samples = wo_stp.shape[0]
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(lr, X_train_tfidf, wo_stp['predicted_category'], cv=cv)


c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


array([0.80013933, 0.79493484, 0.7983362 , 0.80013933, 0.79862306])

# Linear SVM

In [20]:
from sklearn.svm import LinearSVC

In [48]:
df_test['cleaned_hm'] = df_test['cleaned_hm'].apply(lambda x: x.replace('\n',''))
df_test['cleaned_hm'] = df_test['cleaned_hm'].apply(lambda x: x.replace('\r',''))

In [46]:
svc= LinearSVC().fit(X_train_tfidf, wo_stp['predicted_category'])

In [49]:
g = svc.predict(count_vect.transform(df_test['cleaned_hm']))

In [50]:
result = pd.DataFrame()

In [51]:
result['hmid'] = df_test['hmid']
result['predicted_category'] = g

In [52]:
result.head()

,hmid,predicted_category
0,88305,bonding
1,88306,affection
2,88307,affection
3,88308,bonding
4,88309,affection


In [53]:
result.to_csv('result_lsvm_cln_wo.csv')

In [24]:
from sklearn.model_selection import cross_val_score
# scores = cross_val_score(svc, X_train_tfidf, cleaned_df['predicted_category'], cv=5)
# scores

In [122]:
scores = cross_val_score(svc, X_train_tfidf, df_train['predicted_category'], cv=5, scoring='f1_weighted')
scores

array([0.93739333, 0.92899869, 0.93213104, 0.92359722, 0.9244372 ])

In [94]:
from sklearn.model_selection import ShuffleSplit
n_samples = wo_stp.shape[0]
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(svc, X_train_tfidf, wo_stp['predicted_category'], cv=cv)

array([0.92838592, 0.93203293, 0.92473891, 0.93103829, 0.93026468])

In [41]:
from sklearn.model_selection import StratifiedShuffleSplit
n_samples = wo_stp.shape[0]
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(svc, X_train_tfidf, wo_stp['predicted_category'], cv=cv)


array([0.81017949, 0.80665519, 0.80599951, 0.80591755, 0.8053848 ])

# Random Forest

In [44]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)

In [130]:
rf= RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0).fit(X_train_tfidf, cleaned_df['predicted_category'])

In [131]:
g= rf.predict(count_vect.transform(df_test['cleaned_hm']))

In [132]:
result = pd.DataFrame()

In [133]:
result['hmid'] = df_test['hmid']
result['predicted_category'] = g

In [134]:
result.head()

,hmid,predicted_category
0,88305,affection
1,88306,affection
2,88307,affection
3,88308,affection
4,88309,affection


In [135]:
result.to_csv('result_RF_cln.csv')

In [116]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, X_train_tfidf, df_train['predicted_category'], cv=5)
scores

array([0.60470705, 0.6031825 , 0.6051061 , 0.61013015, 0.61628389])

In [123]:
scores = cross_val_score(rf, X_train_tfidf, df_train['predicted_category'], cv=5, scoring='f1_weighted')
scores

c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\adnan\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\adnan\a

array([0.49124443, 0.48904878, 0.49039863, 0.49714242, 0.49984907])

In [108]:
from sklearn.model_selection import ShuffleSplit
n_samples = df_train.shape[0]
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(rf, X_train_tfidf, df_train['predicted_category'], cv=cv)

array([0.60722772, 0.60120462, 0.61905288, 0.58943471, 0.6071172 ])

In [45]:
from sklearn.model_selection import StratifiedShuffleSplit
n_samples = wo_stp.shape[0]
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(rf, X_train_tfidf, wo_stp['predicted_category'], cv=cv)

array([0.37636259, 0.37837063, 0.37603475, 0.37595279, 0.37595279])

# SVM


In [1]:
from sklearn.neighbors import KNeighborsClassifier

In [137]:
knn = KNeighborsClassifier()

In [ ]:
scores = cross_val_score(knn, X_train_tfidf, cleaned_df['predicted_category'], cv=5, scoring='f1_weighted')
scores

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
n_samples = df_train.shape[0]
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(svc1, X_train_tfidf, df_train['predicted_category'], cv=cv)